# Проект по АвтОбрЕЯ

Ccылка на файл с требованиями: https://docs.google.com/document/d/1hHPc2y_fXNvlh623_5GTAMqcAtSucxNaC9ZRuwHi3Wk/edit#

### Над проектом работали:

*   Андрей Зырянов
*   Настя Каприелова
*   Настя Кромина
*   Петрова Даша

При выполнении задачи приветствуются эксперименты с теми методами, про которые рассказывали на занятиях: различные меры ассоциативной связи (коллокационные меры), разрешения семантической неоднозначности (AdaGram, семантическое расстояние по РуТез), определения семантической близости, тематическое моделирование, извлечение словаря тональной лексики…
Для морфологической разметки и выделения синтаксических связей можно использовать UDPipe. Кроме того, можно использовать различные дополнительные ресурсы: векторные модели / эмбеддинги (в том числе мультиязычные), любые другие корпуса (например, описания блюд для составления списка блюд)

**Формируем датасет для корпуса SentiRuEval_rest_train**

In [0]:
import re
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
corpus_train_file = open('SentiRuEval_rest_train.xml', 'r', encoding = 'utf-8', errors='ignore')
corpus_train = corpus_train_file.read()
corpus_train_file.close()

In [0]:
texts = re.findall(r'<text>(.*?)</text>', corpus_train)

In [0]:
food = re.findall(r'<food>(.*?)</food>', corpus_train)

In [0]:
service = re.findall(r'<service>(.*?)</service>', corpus_train)

In [0]:
d = {'texts': texts, 'food': food, 'service': service}
df = pd.DataFrame(data=d)

In [9]:
df.head()

,texts,food,service
0,И пускай на меня не обижается наш прославленны...,8,8
1,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,...",9,10
2,"Советую вам уволить Вашего метродотеля Елену, ...",9,1
3,отличный средне вековый интеръер. Приятное обс...,8,9
4,Ужинали в ресторане Баден-Баден 6 марта . Импо...,10,8


In [10]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 7.6MB/s 


In [0]:
#preprocessing texts in the df
import pymorphy2

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

p = re.compile(r'[\W^ ]')
morph = pymorphy2.MorphAnalyzer()

In [0]:
updated_texts = []
for row in df['texts'][:5000]:
  clean = p.sub(' ', row)
  words = clean.split()
  new_text = ' '.join(words)
  updated_texts.append(new_text)

Создадим списки с биграммами и триграммами для проверки сочетаемости слов

In [0]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [morph.parse(t)[0].normal_form for t in tokens]
    return lemmas


def get_ngrams(tokens, n, patterns=None):
    ngrams = []
    for i in range(len(tokens) - (n - 1)):
        ngram = tokens[i:i+n]
        tags = [morph.parse(t)[0].tag.POS for t in ngram]
        if patterns is not None:
            if tags in patterns:
                ngrams.append(ngram)
        else:
            ngrams.append(ngram)
    return ngrams

In [0]:
import time

In [15]:
start_time = time.time()
all_bigrams_adj = []
for text in updated_texts: 
  bi_grams = get_ngrams(normalize(text), n=2, patterns=[['ADJF', 'NOUN']])
  all_bigrams_adj.append(bi_grams)
print("--- %s seconds ---" % (time.time() - start_time))

--- 514.2666051387787 seconds ---


In [0]:
start_time = time.time()
all_trigrams_adv_adj = []
for text in updated_texts: 
  tr_grams_adv_adj = get_ngrams(normalize(text), n=3, patterns=[['ADVB', 'ADJF', 'NOUN']])
  all_trigrams_adv_adj.append(tr_grams_adv_adj)
print("--- %s seconds ---" % (time.time() - start_time))

In [0]:
start_time = time.time()
all_trigrams_adv_verb = []
for text in updated_texts: 
  tr_grams_adv_verb = get_ngrams(normalize(text), n=3, patterns=[['NOUN', 'ADVB', 'INFN']])
  all_trigrams_adv_verb.append(tr_grams_adv_verb)
print("--- %s seconds ---" % (time.time() - start_time))

Создадим seed-ы из данных нам списков слов: Food 0, Food 1, Service 0 или Service 1

In [0]:
Food_file = open('Food_words.txt', 'r', encoding = 'utf-8')
Food = Food_file.read()
Food_file.close()

In [0]:
Food_split = Food.split('\n')

In [0]:
Food0 = []
Food1 = []
for i in Food_split:
  r = i.split('        ')
  if r[2] == '1':
    Food1.append(r[1])
  if r[2] == '0':
    Food0.append(r[1])

In [0]:
Service_file = open('Service_words.txt', 'r', encoding = 'utf-8')
Service = Service_file.read()
Service_file.close()

In [0]:
Service_split = Service.split('\n')

In [0]:
Service0 = []
Service1 = []
for i in Service_split:
  r1 = i.split('\t')
  if r1[2] == '1':
    Service1.append(r1[1])
  if r1[2] == '0':
    Service0.append(r1[1])

In [23]:
print('f0 = ', len(Food0))
print('f1 = ', len(Food1))
print('s0 = ', len(Service0))
print('s1 = ', len(Service1))

f0 =  3
f1 =  17
s0 =  1
s1 =  19


Списки с негативными слова очень маленькие, в сравнении с количеством позитивной лексики. Мы решили нормализировать наборы, в neg-списки с 0 увеличить количество слов до 5, в pos-списках с 1 - уменьшить

In [231]:
print('Old Food0 list\t', Food0)
seed_neg_food = ['плохой','посредственный','отвратительный','сухой','жесткий']
print('New Food0 list\t', seed_neg_food)


Old Food0 list	 ['плохой', 'невкусный', 'посредственно']
New Food0 list	 ['плохой', 'посредственный', 'отвратительный', 'сухой', 'жесткий']


In [271]:
print('Old Service0 list\t', Service0)
seed_neg_serv = ['хамоватый','злой','грубый','рассеянный','медленный']
print('New Service0 list\t', seed_neg_serv)

Old Service0 list	 ['хамоватый']
New Service0 list	 ['хамоватый', 'злой', 'грубый', 'рассеянный', 'медленный']


In [33]:
print('Old Food1 list\t', Food1)
seed_pos_food = ['вкусный','отличный','приятный','разнообразный','свежий']
print('New Food1 list\t', seed_pos_food)

Old Food1 list	 ['вкусный', 'большой', 'прекрасный', 'разнообразный', 'единственный', 'достойный', 'странный', 'отличный', 'горячий', 'сытный', 'свежий', 'великолепный', 'интересный', 'различный', 'необычный', 'приятный', 'понравиться']
New Food1 list	 ['вкусный', 'отличный', 'приятный', 'разнообразный', 'свежий']


In [37]:
print('Old Service1 list\t', Service1)
seed_pos_serv = ['приветливый','хороший','внимательный','отзывчивый','вежливый']
print('New Service1 list\t', seed_pos_serv)

Old Service1 list	 ['приветливый', 'внимательный', 'приятный', 'вежливый', 'хороший', 'ненавязчивый', 'доброжелательный', 'дружелюбный', 'отличный', 'милый', 'гостеприимный', 'качественный', 'отзывчивый', 'радушный', 'красивый', 'душевный', 'веселый', 'понравиться', 'спасибо']
New Service1 list	 ['приветливый', 'хороший', 'внимательный', 'отзывчивый', 'вежливый']


Загружаем модель araneum_none_fasttextcbow_300_5_2018.model, чтобы использовать её эмбеддинги

In [0]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

In [0]:
from numpy import dot
from numpy.linalg import norm

def cossim(v1,v2): # функция подсчета косинусного расстояния
  return dot(v1, v2)/(norm(v1)*norm(v2))

In [43]:
model_file = 'model.model'
model = KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Проверим вектора на примере

In [227]:
query = 'Картошка была прекрасная.'
query_tok = normalize(query)
query_tok

['картошка', 'быть', 'прекрасный']

In [228]:
query_vec = []
for i in query_tok:
  res = i in model.wv
  print(i, ': ', res)
  print(model[i][:5])
  query_vec.append(model[i])

картошка :  True
[-0.01691983 -0.5142729   0.31042105  0.5475326  -0.28163046]
быть :  True
[ 0.1256133  -0.54293644  0.24227035  0.04432327  0.1083925 ]
прекрасный :  True
[ 0.12641275 -0.31853437  0.04751264 -0.06571354  0.1707755 ]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Создадим массивы векторов слов из seed

In [0]:
pos_food_vec = []
for i in seed_pos_food:
  pos_food_vec.append(model[i])

In [0]:
neg_food_vec = []
for i in seed_neg_food:
  neg_food_vec.append(model[i])

In [0]:
pos_serv_vec = []
for i in seed_pos_serv:
  pos_serv_vec.append(model[i])

In [0]:
neg_serv_vec = []
for i in seed_neg_serv:
  neg_serv_vec.append(model[i])

Создадим также два дополнительных сида со словами категорий еда/сервис, а также списки их векторов

In [0]:
seed_food = ['мясо','рыба','лук','гриб', 'помидор']
food_vec = []
for i in seed_food:
  food_vec.append(model[i])

In [0]:
seed_serv = ['официант','обслуживание','сервис','администратор', 'сотрудник']
serv_vec = []
for i in seed_serv:
  serv_vec.append(model[i])

Посмотрим на близость слов из предложения-примера выше со словами из списка еды

In [229]:
for i in range(len(query_vec)):
  total = 0
  v1 = query_vec[i]
  for j in range(len(food_vec)):
    v2 = food_vec[j]
    sim = cossim(v1,v2)
    total += sim
  space = ' '
  print(f'{query_tok[i]:10} & food_seed = {total}')

картошка   & food_seed = 2.6930912733078003
быть       & food_seed = 1.044472575187683
прекрасный & food_seed = 0.7706841230392456


In [0]:
#кандидаты по близости
food_cand = []
serv_cand = []
pos_food_cand = []
neg_food_cand = []
pos_serv_cand = []
neg_serv_cand = []

#их вектора
food_vec_cand = []
serv_vec_cand = []
pos_food_vec_cand = []
neg_food_vec_cand = []
pos_serv_vec_cand = []
neg_serv_vec_cand = []


#continue

Функция detect находит в предложениии кандидатов для пополнения списков seed и выдает результат в соответствующем формате

Для задачи нахождения оценочных слов с каким-то аспектом в предложении, формат вывода отличается, поэтому для его переформатирования используется функция  result_transform

В функции detect мы для каждого слова из предложения смотрим его косинуную близость со словами из сидов, если число близостей, которые > 0.3, превышает 75% от сида, добавляем слово в кандидаты для пополнения сида.

Если в одном предложении встречается слово с аспектом и оценочное слово, пара становится кандидатом для списка оценочных слов.

Все кандидаты далее будут проверяться на основе сравнения с коллокациями.

In [74]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('russian'))

Функция ниже находит кандидатов в списки оценочных слов и возвращает информацию о том, появился ли новый кандидат

In [0]:
def cands(word, seed_list, seed_vec_list, cand_list, cand_vec_list):
  sim_pass = 0
  old_len = len(cand_list)
  sim_total = 0
  result = [False]
  for p in seed_vec_list: #цикл по сиду векторов оценочных слов
    sim = cossim(model[word], p)
    sim_total += sim
    if sim > 0.3:
      sim_pass += 1
  if sim_pass > 3: #and word not in cand_list and word not in seed_list
    if morph.parse(word)[0].tag.POS == "ADJF":
      result = [True]
      result.append(sim_total)
      #cand_list.append(word)
      #cand_vec_list.append(model[word])

  new_len = len(cand_list)

  return result

In [0]:
def check(t, res, old_len, new_len):
  if res[0] == True: # and new_len == old_len
    t.append(res[1])
  if res[0] == False:
    t.append(0)

In [0]:
def detect(sent_id, sent): # для одного предложения
  candidates = []
  #длины списков кандидатов до разбора предложения
  old_food_len = len(food_cand)
  old_serv_len = len(serv_cand)

  words = normalize(sent)
  food = 'no food'
  service = 'no service'

  for i in range(len(words)):
    pos_tag = morph.parse(words[i])[0].tag.POS
    if words[i] not in stopwords:
      sim_pass = 0
      for j in food_vec:  #цикл для еды: смотрим на близость текущего слова со словами из seed_food
        sim = cossim(model[words[i]], j)
        if sim > 0.3:
          sim_pass += 1
      if sim_pass > 3: #смотрим на количество близких слов: если их больше 3, считаем, что слово относится к еде
        food = 'detected'
        if words[i] not in food_cand and words[i] not in seed_food and pos_tag == "NOUN": #добавляем его в список кандидатов, если его там нет (и его вектор)
          food_cand.append(words[i])
          food_vec_cand.append(model[words[i]])
      new_food_len = len(food_cand) #записываем новую длину, чтобы потом проверить были ли добавлены слова

      sim_pass_serv = 0
      for s in serv_vec: #цикл для сервиса (такой же как для еды)
        sim = cossim(model[words[i]], s)
        if sim > 0.3:
          sim_pass_serv += 1
      if sim_pass_serv > 3:
        service = 'detected'
        if words[i] not in serv_cand and words[i] not in seed_serv and pos_tag == "NOUN": 
          serv_cand.append(words[i])
          serv_vec_cand.append(model[words[i]])
      new_serv_len = len(serv_cand)

      pos_food_res = cands(words[i], seed_pos_food, pos_food_vec, pos_food_cand, pos_food_vec_cand)
      pos_serv_res = cands(words[i], seed_pos_serv, pos_serv_vec, pos_serv_cand, pos_serv_vec_cand)
      neg_food_res = cands(words[i], seed_neg_food, neg_food_vec, neg_food_cand, neg_food_vec_cand)
      neg_serv_res = cands(words[i], seed_neg_serv, neg_serv_vec, neg_serv_cand, neg_serv_vec_cand)

      t = []
      check(t, pos_food_res, old_serv_len, new_serv_len)
      check(t, pos_serv_res, old_food_len, new_food_len)
      check(t, neg_food_res, old_serv_len, new_serv_len)
      check(t, neg_serv_res, old_food_len, new_food_len)

      dop = 0
      if len(t) < 4:
        dop = 4 - len(t)
      for q in range(dop):
        t.append(0)

      sum1 = t[0]+t[1]
      sum2 = t[2]+t[3]

      if sum1 > sum2:
        if t[0] > t[1]:
          c = [sent_id, 'Food', words[i], '1']
          candidates.append(c)
          pos_food_cand.append(words[i])
        if t[0] < t[1]:
          c = [sent_id, 'Service', words[i], '1']
          candidates.append(c)
          pos_serv_cand.append(words[i])
      if sum1 < sum2:
        if t[2] > t[3]:
          c = [sent_id, 'Food', words[i], '0']
          candidates.append(c)
          neg_food_cand.append(words[i])
        if t[2] < t[3]:
          c = [sent_id, 'Service', words[i], '0']
          candidates.append(c)
          neg_serv_cand.append(words[i])

  return candidates

Проверим функции на примере

In [0]:
sent_tt = 'Котлеты были сочные.'

In [600]:
res_tt = detect(1,sent_tt)
res_tt

[[1, 'Food', 'сочный', '1']]

In [601]:
print(food_cand, serv_cand, pos_food_cand, neg_food_cand, pos_serv_cand, neg_serv_cand)

['котлета'] [] ['сочный'] [] [] []


In [0]:
def output(mas):
  res = []
  if mas != []:
    for i in mas:
      s = f'{i[1]}\t{i[2]}\t{i[3]}'
      res.append(s)
  return res

In [603]:
output(res_tt)

['Food\tсочный\t1']

Для нахождения корректного номера токена мы решили распарсить тексты udpipe-ом. Функция searchsent ищет номер нужного токена в полученной разметке

In [0]:
import re

with open('sentences_parsed', 'r', encoding = 'utf-8') as f:
    parsed = f.read()
    conllu_texts = parsed.split('SpacesAfter=\\r\\n')
    reviews = []
    for text in conllu_texts:
        sentences_raw = text.split('# sent_id')
        sentences = []
        for sent_raw in sentences_raw[1:]:
            words = sent_raw.split('\n')[2:]
            sentences.append(words)
        reviews.append(sentences)

def searchsent(i_rev, i_sent, word):
    i_out = []
    lemma = normalize(word)[0]
    for line in reviews[i_rev][i_sent]:
        if re.findall(lemma, line) != []:
            i_out.append(line.split('\t')[0])
    return i_out

In [0]:
#def transform(n, mas): #функция для изменения формата вывода по образцу ручной разметки, однако она не работает
#  res = []
#  for i in mas:
#    sent_id = i[0]
#    token_id = searchsent(n, sent_id, i[2])
#    print('token_id = ',token_id)
#    s = f'{sent_id}\t{token_id}\t{i[1]}\t{i[3]}'
#    print('s = ', s)
#    res.append(s)
#  return res

In [0]:
def text_test(n, text): #detect для текста
  sents = []
  results = []
  results_2 = []
  pattern = re.compile(r'([А-ЯA-Z]((т.п.|т.д.|пр.|г.)|[^?!.\(]|\([^\)]*\))*[.?!]*)')
  for i,sent in enumerate(pattern.findall(text)):
    sents.append(sent[0])
  for i in range(len(sents)):
    result = detect(i, sents[i])
    formarted_result = output(result)
    if formarted_result != []:
      results.append(formarted_result)
    #other_result = transform(n,result)
    #if other_result != []:
    #  results_2.append(other_result)
    ##texts_res.append(results)
  return results#, results_2

In [606]:
test_text = 'Еда была очень вкусной. Официантка была злой и грубой!'
test_res = text_test(0,test_text)
test_res

[['Food\tвкусный\t1'], ['Service\tзлой\t0', 'Service\tгрубый\t0']]

Выделим кандидатов из 5000 текстов

In [0]:
all_res = []
tests_res = []
for i in range(5000):
  try:
    r = text_test(i, texts[i])
    if r != []:
      all_res.append(r)
  except AttributeError:
    continue

In [608]:
len(all_res)

442

In [609]:
for i in all_res[:10]:
  print(i)

[['Service\tуютный\t1']]
[['Food\tотвратительный\t0']]
[['Food\tприятный\t1'], ['Service\tдовольный\t1', 'Service\tтерпеливый\t1', 'Service\tвежливый\t1'], ['Service\tдовольный\t1']]
[['Food\tвкусный\t1'], ['Service\tдовольный\t1']]
[['Service\tдобрый\t1']]
[['Service\tхороший\t1'], ['Service\tвнимательный\t1', 'Service\tучтивый\t1', 'Service\tнавязчивый\t1'], ['Service\tхороший\t1']]
[['Service\tкапризный\t0']]
[['Service\tдовольный\t1'], ['Food\tпревосходный\t1'], ['Service\tхороший\t1']]
[['Food\tпрекрасный\t1', 'Food\tзамечательный\t1']]
[['Food\tприятный\t1'], ['Food\tзамечательный\t1']]


Не будем записывать в файл все слова, возьмем только 50, чтобы посмотреть, корректные ли там выражения

In [0]:
f = open('dict_add.txt', 'a', encoding = 'utf-8')
for i in all_res[:50]:
  for j in i:
    for w in j:
      f.write(w+'\n')
f.close()

Дополнительно можно было бы проверить выделенные слова, изучив коллокации и сравнив контексты употребления слов с списками слов food/service, которые мы автоматически расширили по косинусной близости

Вот насколько увеличились списки:

In [611]:
print('Количество кандидатов для аспекта еды: ', len(food_cand))

Количество кандидатов для аспекта еды:  326


In [612]:
food_cand[:10]

['котлета',
 'еда',
 'блюдо',
 'суп',
 'щи',
 'лапша',
 'пиво',
 'шашлык',
 'жаркое',
 'чебурек']

In [613]:
print('Количество кандидатов для аспекта сервиса: ', len(serv_cand))

Количество кандидатов для аспекта сервиса:  98


In [614]:
serv_cand[:10]

['официантка',
 'ресторан',
 'профи',
 'заведение',
 'клиент',
 'бизнес',
 'интерьер',
 'заказ',
 'посетитель',
 'шеф']